In [1]:
!gdown 1gj5-HatoF4n6V_RYB2r9vYpeawJvgf2-
!gdown 17twDjhPHlpV9ONl_Rekg_BuFLdnZLUnq

Downloading...
From: https://drive.google.com/uc?id=1gj5-HatoF4n6V_RYB2r9vYpeawJvgf2-
To: /Users/tada/Documents/GitHub/data-warehouse-mining/src/assgn01_etlAndDataProcessing/ETLDataSource1-286218-17243904498090.xlsx
100%|██████████████████████████████████████| 15.3k/15.3k [00:00<00:00, 21.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=17twDjhPHlpV9ONl_Rekg_BuFLdnZLUnq
To: /Users/tada/Documents/GitHub/data-warehouse-mining/src/assgn01_etlAndDataProcessing/ETLDataSource2-286218-17243904776879.xlsx
100%|██████████████████████████████████████| 12.7k/12.7k [00:00<00:00, 15.7MB/s]


In [2]:
import pandas as pd
from pandas import DataFrame, Series

rawOrderSource1: DataFrame = pd.read_excel(io='ETLDataSource1-286218-17243904498090.xlsx', sheet_name='orderSource1')
rawProductSource1: DataFrame = pd.read_excel(io='ETLDataSource1-286218-17243904498090.xlsx', sheet_name='productSource1')
rawStateLookupSource1: DataFrame = pd.read_excel(io='ETLDataSource1-286218-17243904498090.xlsx', sheet_name='StateLookup')

rawOrderSource2: DataFrame = pd.read_excel(io='ETLDataSource2-286218-17243904776879.xlsx', sheet_name='orderSource2')
rawProductSource2: DataFrame = pd.read_excel(io='ETLDataSource2-286218-17243904776879.xlsx', sheet_name='productSource2')

In [3]:
stateLookup: dict[str, str] = {
    abbreviation: stateName
    for _, stateName, abbreviation
    in rawStateLookupSource1.itertuples()
}

statusLookup: dict[int, str] = {
    1: 'Silver',
    2: 'Gold',
    3: 'Platinum',
}

def identity(x):
    return x

def getFirstName(fullName: str) -> str:
    return fullName.split()[0]

def getLastName(fullName: str) -> str:
    return fullName.split()[-1]

def removePrefixA(text: str) -> int:
    return int(text.upper().replace('A', ''))

namesSource1: DataFrame = (
    rawOrderSource1['CustomerName']
    .transform(func=[identity, getFirstName, getLastName])
    .rename(columns={'identity': 'CustomerName', 'getFirstName': 'CustomerFirstName', 'getLastName': 'CustomerLastName'})
)

cleanedOrderIdSource2: DataFrame = (
    rawOrderSource2['OrderID']
    .transform(func=[identity, removePrefixA])
    .rename(columns={'identity': 'OrderID', 'removePrefixA': 'newOrderID'})
)

totalDiscountSource2: Series = (
    (rawProductSource2['FullPrice'] - rawProductSource2['ExtendedPrice'])
    .rename(index='TotalDiscount')
)


In [4]:
processedSource1: DataFrame = (
    rawOrderSource1
    .merge(right=rawProductSource1, how='inner', on='OrderID')
    .replace(to_replace={'CustomerState': stateLookup})
    .merge(right=namesSource1, how='inner', on='CustomerName')
    .drop(columns='CustomerName')
    .sort_index(axis='columns')
)

processedSource2: DataFrame = (
    rawOrderSource2
    .merge(right=rawProductSource2, how='inner', on='OrderID')
    .merge(right=cleanedOrderIdSource2, how='inner', on='OrderID')
    .drop(columns='OrderID')
    .rename(columns={'newOrderID': 'OrderID'})
    .replace(to_replace={'CustomerStatus': statusLookup})
    .drop(columns='TotalDiscount')
    .merge(right=totalDiscountSource2, left_index=True, right_index=True)
    .sort_index(axis='columns')
)

In [5]:
combinedSource: DataFrame = pd.concat([processedSource1, processedSource2])
combinedSource['CustomerName'] = combinedSource['CustomerFirstName'] + ' ' + combinedSource['CustomerLastName']
combinedSource = (
    combinedSource
    .drop(columns=['CustomerFirstName', 'CustomerLastName'])
    .sort_index(axis='columns')
)

combinedSource.to_csv(
    path_or_buf='pandasEtl.csv',
    index=False
)